In [2]:
%reload_ext autoreload
%autoreload 2
#show all columnsdf
#ignore warnings.

In [7]:
import pandas as pd
pd.set_option('display.max_columns', 9000)
import numpy as np
import os, sys
import importlib
import yaml, json
from pathlib import Path
base=Path('../Data/patents/assignees')
inst=Path('../Data/institutions/')


In [7]:
import pandas as pd
from pathlib import Path

# Define base directory
base = Path('../Data/patent/assignee')

# Get all CSV files in the directory
csv_files = list(base.glob("*.csv"))

# Dictionary to store sampled dataframes
sheets = {}

# Load a sample of 1000 records from each CSV
for file in csv_files:
    try:
        df = pd.read_csv(file, nrows=1000)
        sheets[file.stem] = df  # Store dataframe with file name (without extension) as key
    except Exception as e:
        print(f"Error reading {file}: {e}")


In [8]:
df=pd.read_csv(inst/'institutions.csv')
ass = Path('../Data/patent/assignee/assignee.csv')
df_a=pd.read_csv(ass)
df

,Institution_OrgUEINum_C,Sum,Count,Institution_Name_C
0,GNJ7BBP73WE9,2.013853e+09,4689,Regents of the University of Michigan - Ann Arbor
1,HD1WMN6945W6,2.095012e+09,4214,University of Washington
2,Y8CWNJRCNN91,2.623375e+09,4069,University of Illinois at Urbana-Champaign
3,NPM2J7MSCF61,1.487659e+09,3861,Pennsylvania State Univ University Park
4,GS3YEVSS12N6,1.879913e+09,3758,University of California-Berkeley
...,...,...,...,...
17400,Butler Annie,0.000000e+00,0,Butler Annie
17401,Ray Emily A,0.000000e+00,0,Ray Emily A
17402,Herbold Craig W,0.000000e+00,0,Herbold Craig W
17403,Swedish Health Services,0.000000e+00,0,Swedish Health Services


In [18]:
df_ee=df_a.value_counts('ee_name')
df_ee=df_ee.reset_index()
df_ee.sort_values('ee_name',inplace=True)

In [19]:
df_ee.to_csv('../Data/patents/entities.csv')

In [13]:
df_ee.reset_index(, inplace=True)

#df_ee.sort_values('ee_name')

In [14]:

from sentence_transformers import SentenceTransformer
import torch
import pandas as pd

def load_model():
    """Load a pre-trained SentenceTransformer model."""
    return SentenceTransformer('all-MiniLM-L6-v2')

def encode_texts(model, texts):
    """Encode a list of texts into embeddings using a SentenceTransformer model."""
    return model.encode(texts, convert_to_tensor=True)

def find_closest_match(name, model, search_space_embeddings, search_space_df, return_columns):
    """Find the closest match for a given name using cosine similarity."""
    query_embedding = model.encode(name, convert_to_tensor=True)
    similarities = torch.nn.functional.cosine_similarity(query_embedding, search_space_embeddings, dim=1)
    closest_idx = torch.argmax(similarities).item()
    
    result = {'Similarity_Score': similarities[closest_idx].item()}
    for col in return_columns:
        result[col] = search_space_df.iloc[closest_idx][col]
    
    return result

# Load model
model = load_model()

# Load and encode institution names in the search space
cs_embeddings = encode_texts(model, cs['INSTNM'].tolist())

# Define columns to return
return_columns = ['INSTNM', 'UNITID', 'OPEID', 'OPEID6']

# Initialize results storage
results = []
batch_size = 100  # Save progress every 100 records

# Iterate over the input DataFrame and find closest matches
for i, (name, org_uei) in enumerate(zip(df['Institution_Name_C'], df['Institution_OrgUEINum_C']), start=1):
    match_result = find_closest_match(name, model, cs_embeddings, cs, return_columns)
    match_result.update({'Institution_Name_C': name, 'Institution_OrgUEINum_C': org_uei, 'Similarity_Score': match_result['Similarity_Score']})
    results.append(match_result)
    
    # Save intermediate results
    if i % batch_size == 0:
        print(f"Processed {i} records, saving progress...")
        pd.DataFrame(results).to_csv(f"progress_{i}.csv", index=False)

# Final save
final_df = pd.DataFrame(results)
final_df.to_csv("final_results.csv", index=False)

# Merge results into the original DataFrame
df = df.merge(final_df, on=['Institution_Name_C', 'Institution_OrgUEINum_C'], how='left')


0          164781
1          130758
2           75652
3           59364
4           58294
            ...  
1321777         1
1321778         1
1321779         1
1321780         1
1321781         1
Name: count, Length: 1321782, dtype: int64

In [9]:

# Define function to write to Excel
def write_excel(outpath, outfile, sheets):
    print("Outputting final Excel file:", str(outfile))
    with pd.ExcelWriter(outpath / outfile) as writer:
        for key, value in sheets.items():
            if len(value) < 1048576:
                print("Outputting sheet:", key)
                value = value.apply(pd.to_numeric, errors='ignore')
                value.to_excel(writer, sheet_name=key, index=True)
            else:
                print("Too large, saving CSV:", key)
                value.to_csv(outpath / f"{key}.csv", index=False)

# Define output path and file name
output_path = Path('../Data/patents/assignees/')
output_file = 'sampled_assignees.xlsx'

# Ensure output directory exists
output_path.mkdir(parents=True, exist_ok=True)

# Write to Excel
write_excel(output_path, output_file, sheets)

/var/folders/bv/2rb8dc7100ncxqhclz10zf5c0000gn/T/ipykernel_61981/756587973.py:8: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  value = value.apply(pd.to_numeric, errors='ignore')
Exception ignored in: <function ZipFile.__del__ at 0x102d06a70>
Traceback (most recent call last):
  File "/Users/jasonkuruzovich/anaconda3/envs/newdeep/lib/python3.10/zipfile.py", line 1834, in __del__
    self.close()
  File "/Users/jasonkuruzovich/anaconda3/envs/newdeep/lib/python3.10/zipfile.py", line 1851, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


Outputting final Excel file: sampled_assignees.xlsx
Outputting sheet: assignment
Outputting sheet: assignee
Outputting sheet: assignor
Outputting sheet: documentid
Outputting sheet: documentid_admin
Outputting sheet: assignment_conveyance


In [ ]:
import os
import requests
from tqdm import tqdm
from zipfile import ZipFile, BadZipFile
import zipfile
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime, timedelta
import time

# Function to generate file names based on start and end years
def generate_file_names(start_year, end_year):
    file_names = []
    for year in range(start_year, end_year + 1):
        start_date = datetime(year, 1, 1)
        days_to_tuesday = (1 - start_date.weekday() + 7) % 7 
        tuesday = start_date + timedelta(days=days_to_tuesday)

        # Iterate through all Tuesdays of the year
        while tuesday.year == year:
            date_str = tuesday.strftime('%y%m%d')
            file_name = f"ipg{date_str}.zip"
            file_names.append(f"{year}/{file_name}")
            tuesday += timedelta(days=7)

    return file_names

start_year = 2009
end_year = 2024
filenames = generate_file_names(start_year, end_year)

# Create output directory for downloaded files
output_dir = '../Data/patent/fullpatentdata'
os.makedirs(output_dir, exist_ok=True)

base_url = 'https://bulkdata.uspto.gov/data/patent/grant/redbook/fulltext/'

# Function to download files from the USPTO website with retries
def download_file(file, max_retries=3):
    try:
        full_url = base_url + file
        headers = {'User-Agent': 'Mozilla/5.0'}
        file_path = os.path.join(output_dir, os.path.basename(file))

        for attempt in range(max_retries):
            response = requests.get(full_url, headers=headers, stream=True)
            if response.status_code == 200:
                # Check if the response content is a valid ZIP file
                if response.headers.get('Content-Type') != 'application/zip':
                    print(f"Invalid file type for {file}")
                    return

                with open(file_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=128):
                        f.write(chunk)
                return
            else:
                print(f"Attempt {attempt + 1}: Failed to download {file} - HTTP {response.status_code}")
                time.sleep(2)

    except Exception as e:
        print(f"Error downloading {file}: {e}")

# Download files using ThreadPoolExecutor for multiprocessing
with ThreadPoolExecutor(max_workers=10) as executor:
    list(tqdm(executor.map(download_file, filenames), total=len(filenames)))

# Extracting and cleaning up zip files in parallel
def extract_zip(filename):
    zip_path = os.path.join(output_dir, filename)
    try:
        with ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(output_dir)
        os.remove(zip_path)
    except BadZipFile:
        print(f"Error: The file {filename} is not a valid zip file or is corrupted.")

zip_files = [f for f in os.listdir(output_dir) if f.endswith(".zip")]

with ThreadPoolExecutor(max_workers=10) as executor:
    list(tqdm(executor.map(extract_zip, zip_files), total=len(zip_files)))

print("Download and extraction process completed.")


 49%|████▉     | 410/835 [57:15<2:55:35, 24.79s/it]

Error downloading 2016/ipg161227.zip: ('Connection broken: IncompleteRead(41973011 bytes read, 82006373 more expected)', IncompleteRead(41973011 bytes read, 82006373 more expected))


 49%|████▉     | 411/835 [58:34<4:35:19, 38.96s/it]

Error downloading 2016/ipg161115.zip: ('Connection broken: IncompleteRead(105838161 bytes read, 29932021 more expected)', IncompleteRead(105838161 bytes read, 29932021 more expected))
Error downloading 2017/ipg170103.zip: ('Connection broken: IncompleteRead(38100938 bytes read, 79784179 more expected)', IncompleteRead(38100938 bytes read, 79784179 more expected))
Error downloading 2016/ipg161213.zip: ('Connection broken: IncompleteRead(90583436 bytes read, 7707146 more expected)', IncompleteRead(90583436 bytes read, 7707146 more expected))
Error downloading 2016/ipg161220.zip: ('Connection broken: IncompleteRead(75239300 bytes read, 8861054 more expected)', IncompleteRead(75239300 bytes read, 8861054 more expected))
Error downloading 2017/ipg170110.zip: ('Connection broken: IncompleteRead(24453418 bytes read, 101837122 more expected)', IncompleteRead(24453418 bytes read, 101837122 more expected))
Error downloading 2016/ipg161206.zip: ('Connection broken: IncompleteRead(111130483 bytes 

 49%|████▉     | 412/835 [1:03:06<11:56:32, 101.64s/it]

Error downloading 2016/ipg161122.zip: ('Connection broken: IncompleteRead(102488842 bytes read, 22635997 more expected)', IncompleteRead(102488842 bytes read, 22635997 more expected))
Error downloading 2017/ipg170307.zip: ('Connection broken: IncompleteRead(5536666 bytes read, 125376512 more expected)', IncompleteRead(5536666 bytes read, 125376512 more expected))
Error downloading 2017/ipg170404.zip: ('Connection broken: IncompleteRead(10422522 bytes read, 99249135 more expected)', IncompleteRead(10422522 bytes read, 99249135 more expected))
Error downloading 2017/ipg170328.zip: ('Connection broken: IncompleteRead(9695546 bytes read, 121275125 more expected)', IncompleteRead(9695546 bytes read, 121275125 more expected))
Error downloading 2017/ipg170418.zip: ('Connection broken: IncompleteRead(29727266 bytes read, 108146572 more expected)', IncompleteRead(29727266 bytes read, 108146572 more expected))
Error downloading 2017/ipg170425.zip: ('Connection broken: IncompleteRead(4386947 byte

 50%|█████     | 420/835 [11:28:07<11:19:55, 98.30s/it]
